In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
from crewai import LLM

llm = LLM(
    model="groq/llama-3.1-70b-versatile", 
    api_key="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"
)

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': '../configs/basic_config/agents.yaml',
    'tasks': '../configs/basic_config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [7]:
# Creating Agents
onboarding_agent = Agent(
  config=agents_config['onboarding_agent'],
  llm=llm,
  human_input=True
)

question_creation_agent = Agent(
  config=agents_config['question_creation_agent'],
  llm=llm
)

# Creating Tasks
user_onboarding = Task(
  config=tasks_config['user_onboarding'],
  agent=onboarding_agent
)

question_creation = Task(
  config=tasks_config['question_creation'],
  agent=question_creation_agent
)

question_critic = Task(
  config=tasks_config['question_critic'],
  agent=question_creation_agent
)

onboarding_crew = Crew(
  agents=[
    onboarding_agent
  ],
  tasks=[
    user_onboarding
  ],
  verbose=True
)

question_crew = Crew(
  agents=[
    question_creation_agent
  ],
  tasks=[
    question_creation,
    question_critic
  ],
  verbose=True
)

2024-11-09 16:14:03,852 - 8368726272 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2024-11-09 16:14:03,855 - 8368726272 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [9]:
from crewai import Flow
from crewai.flow.flow import listen, start

class fullFlow(Flow):
    @start()
    def user_input(self):
        return {
            'user_input': "Oi! Meu nome é Guilherme, tenho 32 anos de idade. Eu nunca programei antes, mas tenho curiosidade de aprender! Eu gosto de futebol e series."
        }
    
    @listen(user_input)
    def onboarding(self, user_input):
        user_info = onboarding_crew.kickoff(inputs=user_input)
        self.state['user_info'] = user_info

        return user_info
    
    @listen(onboarding)
    def question_creation(self, user_info):
        question = question_crew.kickoff(inputs=user_info)
    
        return question


In [10]:
flow = fullFlow()

In [11]:
flow.plot()

Plot saved as crewai_flow.html


In [16]:
question = await flow.kickoff()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
from

In [6]:
crew.kickoff(inputs=inputs)

# Agent: Be a warm and welcoming onboarding assistant
## Task: Use the input from the user to determine their persona, tastes, and knowledge level. Oi! Meu nome é Guilherme, tenho 32 anos de idade. Eu nunca programei antes, mas tenho curiosidade de aprender! Eu gosto de futebol e series.



# Agent: Be a warm and welcoming onboarding assistant
## Final Answer: 
<user_language> Português </user_language>

<persona> Guilherme é um homem de 32 anos de idade, provavelmente com uma base sólida de ensino fundamental e médio, mas sem experiência prévia em programação. Ele parece ser uma pessoa com uma abordagem simples e direta, seja em sua linguagem ou em suas preferências pessoais. </persona>

<tastes> Guilherme gosta de futebol e séries, o que sugere que ele tem uma preferência por atividades lúdicas e entretenimento. Isso pode indicar que ele aprende melhor através de exemplos práticos e hands-on, com aplicativos e projetos que sejam interessantes e desafiadores. </tastes>

<knowledge_lev

CrewOutput(raw='Com base na análise da questão criada, podemos identificar alguns pontos fortes e fracos. \n\nUm ponto positivo é que a questão está relacionada ao contexto do aplicativo de futebol, o que é relevante para o usuário. Além disso, a questão apresenta várias opções de respostas, o que permite ao usuário escolher a opção que melhor lhe parece adequada.\n\nNo entanto, há alguns pontos que podem ser melhorados. Em primeiro lugar, a questão não é específica o suficiente em relação ao conhecimento do usuário. Embora seja relacionada ao contexto do futebol, não é claro quais habilidades ou conceitos específicos o usuário precisa ter para responder corretamente.\n\nAlém disso, a questão não tem um contexto claro definido para o usuário progredir em seu aprendizado. A questão está mais focada em testar a capacidade do usuário de escolher a opção correta do que em fornecer um desafio que o ajude a aprender ou melhorar suas habilidades.\n\nPara melhorar a questão, seria recomendável